In [20]:
%matplotlib inline
import numpy as np
import pandas as pd
import re as re
import os.path

import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit


# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.callbacks import History,LearningRateScheduler
# from tensorflow.keras.layers import Dropout
# print('TensorFlow %s, Keras %s, numpy %s, pandas %s'%(tf.__version__,keras.__version__, np.__version__,pd.__version__))
__DEBUG__=False

In [41]:
train = pd.read_csv('data_concatener.csv', sep=';')
test  = pd.read_csv('test_set_values.csv')
full_data = [train, test]
finalfile_index=test.id #Index des données de test pour le résultat final

# Useless column
train.drop('num_private', axis=1, inplace=True)
train.drop('date_recorded', axis=1, inplace=True)
train.drop('funder', axis=1, inplace=True)
train.drop('installer', axis=1, inplace=True)
train.drop('scheme_name', axis=1, inplace=True)
train.drop('scheme_management', axis=1, inplace=True)
train.drop('recorded_by', axis=1, inplace=True)
train.drop('latitude', axis=1, inplace=True)
train.drop('longitude', axis=1, inplace=True)
train.drop('wpt_name', axis=1, inplace=True)
train.drop('subvillage', axis=1, inplace=True)
test.drop('num_private', axis=1, inplace=True)
test.drop('date_recorded', axis=1, inplace=True)
test.drop('funder', axis=1, inplace=True)
test.drop('installer', axis=1, inplace=True)
test.drop('scheme_name', axis=1, inplace=True)
test.drop('scheme_management', axis=1, inplace=True)
test.drop('recorded_by', axis=1, inplace=True)
test.drop('latitude', axis=1, inplace=True)
test.drop('longitude', axis=1, inplace=True)
test.drop('subvillage', axis=1, inplace=True)
test.drop('wpt_name', axis=1, inplace=True)

# Double column
train.drop('quantity', axis=1, inplace=True)
train.drop('source', axis=1, inplace=True)
train.drop('waterpoint_type', axis=1, inplace=True)
train.drop('payment', axis=1, inplace=True)
train.drop('management', axis=1, inplace=True)
train.drop('extraction_type', axis=1, inplace=True)
test.drop('quantity', axis=1, inplace=True)
test.drop('source', axis=1, inplace=True)
test.drop('waterpoint_type', axis=1, inplace=True)
test.drop('payment', axis=1, inplace=True)
test.drop('management', axis=1, inplace=True)
test.drop('extraction_type', axis=1, inplace=True)

# Refactoring value empty
for dataset in full_data:
    # dataset.loc[dataset.latitude == '-2,00E-08', 'latitude'] = float('0.0')
    # dataset.loc[dataset.latitude.replace('.',',')]
    # dataset.loc['latitude'] = dataset['latitude'].astype(float)
    dataset.loc[dataset.public_meeting.isnull(), 'public_meeting'] = 0
    dataset.loc[dataset.public_meeting == False, 'public_meeting'] = 0
    dataset.loc[dataset.public_meeting == True, 'public_meeting'] = 1
    # dataset.loc[dataset.funder.isnull(), 'funder'] = 'Unknown'
    # dataset.loc[dataset.installer.isnull(), 'installer'] = 'Unknown'
    # dataset.loc[dataset.subvillage.isnull(), 'subvillage'] = 'None'
    # dataset.loc[dataset.scheme_management.isnull(), 'scheme_management'] = 'Other'
    # dataset.loc[dataset.scheme_name.isnull(), 'scheme_name'] = 'Unknown'
    dataset.loc[dataset.permit.isnull(), 'permit'] = 0
    dataset.loc[dataset.permit == False, 'permit'] = 0
    dataset.loc[dataset.permit == True, 'permit'] = 1
    dataset.loc[dataset.construction_year == 0, 'construction_year'] = np.NaN
    construction_year_avg = dataset['construction_year'].mean() # Calcul de la valeur moyenne
    construction_year_std = dataset['construction_year'].std()  # Calcul de l'écart type
    construction_year_null_count = dataset['construction_year'].isnull().sum() # nombre de valeur nulle
    construction_year_null_random_list = np.random.randint(construction_year_avg - construction_year_std, construction_year_avg + construction_year_std, size=construction_year_null_count)   
    dataset.loc[np.isnan(dataset['construction_year']),'construction_year'] = construction_year_null_random_list    
    dataset['construction_year'] = dataset['construction_year'].astype(int)
    dataset.loc[dataset.management_group == 'unknown', 'management_group'] = 'other'
    dataset.loc[dataset.payment_type == 'unknown', 'payment_type'] = 'other'

    


train['functional']= (train['status_group']=='functional')*1
train['non_functional']= (train['status_group']=='non functional')*1
train['functional_needs_repair']= (train['status_group']=='functional needs repair')*1

BASE_DIR = os.path.abspath('')
print(BASE_DIR)
# train.to_csv(os.path.join(BASE_DIR, 'cc.csv'))
train

C:\Users\Clemster\Documents\Cours\EPSI\projet_ia


,id,amount_tsh,gps_height,basin,region,region_code,district_code,lga,ward,population,...,water_quality,quality_group,quantity_group,source_type,source_class,waterpoint_type_group,status_group,functional,non_functional,functional_needs_repair
0,69572,6000.0,1390,Lake Nyasa,Iringa,11,5,Ludewa,Mundindi,109,...,soft,good,enough,spring,groundwater,communal standpipe,functional,1,0,0
1,8776,0.0,1399,Lake Victoria,Mara,20,2,Serengeti,Natta,280,...,soft,good,insufficient,rainwater harvesting,surface,communal standpipe,functional,1,0,0
2,34310,25.0,686,Pangani,Manyara,21,4,Simanjiro,Ngorika,250,...,soft,good,enough,dam,surface,communal standpipe,functional,1,0,0
3,67743,0.0,263,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,Nanyumbu,58,...,soft,good,dry,borehole,groundwater,communal standpipe,non functional,0,1,0
4,19728,0.0,0,Lake Victoria,Kagera,18,1,Karagwe,Nyakasimbi,0,...,soft,good,seasonal,rainwater harvesting,surface,communal standpipe,functional,1,0,0
5,9944,20.0,0,Pangani,Tanga,4,8,Mkinga,Moa,1,...,salty,salty,enough,other,unknown,communal standpipe,functional,1,0,0
6,19816,0.0,0,Internal,Shinyanga,17,3,Shinyanga Rural,Samuye,0,...,soft,good,enough,borehole,groundwater,hand pump,non functional,0,1,0
7,54551,0.0,0,Lake Tanganyika,Shinyanga,17,3,Kahama,Chambo,0,...,milky,milky,enough,shallow well,groundwater,hand pump,non functional,0,1,0
8,53934,0.0,0,Lake Tanganyika,Tabora,14,6,Tabora Urban,Itetemia,0,...,salty,salty,seasonal,borehole,groundwater,hand pump,non functional,0,1,0
9,46144,0.0,0,Lake Victoria,Kagera,18,1,Karagwe,Kaisho,0,...,soft,good,enough,shallow well,groundwater,hand pump,functional,1,0,0


In [27]:
print (train[["quality_group", "functional"]].groupby(['quality_group'], as_index=1).mean())
print (train[["quality_group", "non_functional"]].groupby(['quality_group'], as_index=1).mean())
print (train[["quality_group", "functional_needs_repair"]].groupby(['quality_group'], as_index=1).mean())

print('------------------------------------------------')

print (train[["quantity_group", "functional"]].groupby(['quantity_group'], as_index=1).mean())
print (train[["quantity_group", "non_functional"]].groupby(['quantity_group'], as_index=1).mean())
print (train[["quantity_group", "functional_needs_repair"]].groupby(['quantity_group'], as_index=1).mean())

print('------------------------------------------------')

print (train[["waterpoint_type_group", "functional"]].groupby(['waterpoint_type_group'], as_index=1).mean())
print (train[["waterpoint_type_group", "non_functional"]].groupby(['waterpoint_type_group'], as_index=1).mean())
print (train[["waterpoint_type_group", "functional_needs_repair"]].groupby(['waterpoint_type_group'], as_index=1).mean())

print('------------------------------------------------')

print (train[["management_group", "functional"]].groupby(['management_group'], as_index=1).mean())
print (train[["management_group", "non_functional"]].groupby(['management_group'], as_index=1).mean())
print (train[["management_group", "functional_needs_repair"]].groupby(['management_group'], as_index=1).mean())

print('------------------------------------------------')

print (train[["payment_type", "functional"]].groupby(['payment_type'], as_index=1).mean())
print (train[["payment_type", "non_functional"]].groupby(['payment_type'], as_index=1).mean())
print (train[["payment_type", "functional_needs_repair"]].groupby(['payment_type'], as_index=1).mean())

print('------------------------------------------------')

print (train[["region", "functional"]].groupby(['region'], as_index=1).mean())
print (train[["region", "non_functional"]].groupby(['region'], as_index=1).mean())
print (train[["region", "functional_needs_repair"]].groupby(['region'], as_index=1).mean())


# print (train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split

# rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)
# rnd_clf.fit(X_train, y_train)

               functional
quality_group            
colored          0.502041
fluoride         0.723502
good             0.565941
milky            0.544776
salty            0.460828
unknown          0.140725
               non_functional
quality_group                
colored              0.387755
fluoride             0.216590
good                 0.357236
milky                0.437811
salty                0.482002
unknown              0.840618
               functional_needs_repair
quality_group                         
colored                       0.110204
fluoride                      0.059908
good                          0.076823
milky                         0.017413
salty                         0.057170
unknown                       0.018657
------------------------------------------------
                functional
quantity_group            
dry               0.025136
enough            0.652323
insufficient      0.523234
seasonal          0.574074
unknown           0.269962
  

In [42]:
from sklearn.model_selection import train_test_split
#Séparation des valeurs de train et label (tous les exemples)
X_alltrain = train.values[:, 1:]
y_alltrain = train.values[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X_alltrain, y_alltrain, random_state=42)
print('%i X_train, %i X_test, %i y_train, %i y_test'%(
    X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]))
print('%i X_alltrain, %i y_alltrain'%(X_alltrain.shape[0], y_alltrain.shape[0]))
feature_names=train.columns.tolist()[1:-4]
target_names=["Fonctionnel","Non fonctionnel","Necessite réparation"]
print('features:',feature_names)
print('target:',target_names)

44550 X_train, 14850 X_test, 44550 y_train, 14850 y_test
59400 X_alltrain, 59400 y_alltrain
features: ['amount_tsh', 'gps_height', 'basin', 'region', 'region_code', 'district_code', 'lga', 'ward', 'population', 'public_meeting', 'permit', 'construction_year', 'extraction_type_group', 'extraction_type_class', 'management_group', 'payment_type', 'water_quality', 'quality_group', 'quantity_group', 'source_type', 'source_class', 'waterpoint_type_group']
target: ['Fonctionnel', 'Non fonctionnel', 'Necessite réparation']


In [43]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
def plot_decision_boundary(clf,X,y, axes=[-0, 5, 0, 550], axis_name=['x1','x2'],alpha=0.5, contour=True):
    """
    Fonction pour l'affichage 2 D des résultats   
    
    Parameters
    ----------
    clf : Classifier à afficher
    X : features de Données a afficher
    y : labels de Données a afficher  
    axes : : Tailles des axes (valeur min/max)
    axis_name : Nom des axes sur le graphique
    alpha : Transparence des points
    contour : Afichage du contour
    """     
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bo",label="Disparu", alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "ys", label="Rescapé",alpha=alpha)
    plt.axis(axes)
    plt.xlabel(axis_name[0], fontsize=18)
    plt.ylabel(axis_name[1]+ "  ",fontsize=18, rotation=0)    
    plt.legend(loc="lower right", fontsize=14)

import os
from IPython.display import Image

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import accuracy_score

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X_train, y_train)

export_graphviz(
        tree_clf,
        out_file="train_tree.dot",
        feature_names=feature_names,
        class_names=target_names,
        rounded=True,
        filled=True
    )

os.system("dot -Tpng train_tree.dot -o train_tree.png") 
Image("train_tree.png")

ValueError: could not convert string to float: 'Wami / Ruvu'